In [1]:
import numpy as np
import cv2

In [2]:
import pickle

with open('Train/X_train_final.pkl', 'rb') as f:
    X = pickle.load(f)
    X = np.array(X)
#     X = np.expand_dims(X, axis=3)
    X = X/255.
with open('Train/y_train_final.pkl', 'rb') as f:
    y = pickle.load(f)
with open('Train/name_map_final.pkl', 'rb') as f:
    names = pickle.load(f)


In [3]:
np.shape(X), np.shape(y), np.shape(names)


((400, 250, 250, 3), (400,), (10,))

In [37]:
from keras.models import Sequential, Model
from keras.layers import *
from keras.optimizers import *
from keras import applications

In [49]:
import tensorflow as tf
from keras import backend as K
import h5py


def convnet_model_():
    vgg_model = applications.VGG16(weights=None, include_top=False, input_shape=(221, 221, 3))
    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
#    x = Lambda(lambda x_: K.l2_normalize(x,axis=1))(x)
    x = Lambda(K.l2_normalize)(x)
    convnet_model = Model(inputs=vgg_model.input, outputs=x)
    return convnet_model

def deep_rank_model():
    convnet_model = convnet_model_()

    first_input = Input(shape=(221, 221, 3))
    first_conv = Conv2D(96, kernel_size=(8,8), strides=(16,16), padding='same')(first_input)
    first_max = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(first_conv)
    first_max = Flatten()(first_max)
#     first_max = Lambda(K.l2_normalize)(first_max)
    first_max = Lambda(lambda x: K.l2_normalize(x, axis=1))(first_max)

    second_input = Input(shape=(221, 221, 3))
    second_conv = Conv2D(96, kernel_size=(8,8), strides=(32,32), padding='same')(second_input)
    second_max = MaxPool2D(pool_size=(7,7), strides=(4,4), padding='same')(second_conv)
    second_max = Flatten()(second_max)
    second_max = Lambda(lambda x: K.l2_normalize(x, axis=1))(second_max)
#     second_max = Lambda(K.l2_normalize)(second_max)

    merge_one = concatenate([first_max, second_max])
    merge_two = concatenate([merge_one, convnet_model.output])
    emb = Dense(4096)(merge_two)
    emb = Dense(128)(emb)
    l2_norm_final = Lambda(lambda x: K.l2_normalize(x, axis=1))(emb)
#     l2_norm_final = Lambda(K.l2_normalize)(emb)

    final_model = Model(inputs=[first_input, second_input, convnet_model.input], outputs=l2_norm_final)

    return final_model

In [50]:
deep_rank_model = deep_rank_model()
deep_rank_model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 221, 221, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 221, 221, 64) 1792        input_11[0][0]                   
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 221, 221, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 110, 110, 64) 0           block1_conv2[0][0]               
____________________________________________________________________________________________

In [51]:
batch_size = 24

_EPSILON = K.epsilon()
def _loss_tensor(y_true, y_pred):
    y_pred = K.clip(y_pred, _EPSILON, 1.0 - _EPSILON)
    loss = 0.
    g = 1.
    for i in range(0, batch_size, 3):
        try:
            q_embedding = y_pred[i]
            p_embedding = y_pred[i+1]
            n_embedding = y_pred[i+2]
            D_q_p = K.sqrt(K.sum((q_embedding - p_embedding)**2))
            D_q_n = K.sqrt(K.sum((q_embedding - n_embedding)**2))
            loss = loss + g + D_q_p - D_q_n
        except:
            continue
    loss = loss/batch_size*3
    return K.maximum(loss, 0)

In [52]:
deep_rank_model.compile(loss=_loss_tensor, optimizer=SGD(lr=0.001, momentum=0.9, nesterov=True))


In [53]:
def image_batch_generator(images, labels, batch_size):
    labels = np.array(labels)
    while True:
        batch_paths = np.random.choice(a = len(images), size = batch_size//3)
        input_1 = []

        for i in batch_paths:
            pos = np.where(labels == labels[i])[0]
            neg = np.where(labels != labels[i])[0]

            j = np.random.choice(pos)
            while j == i:
                j = np.random.choice(pos)

            k = np.random.choice(neg)
            while k == i:
                k = np.random.choice(neg)

            input_1.append(images[i])
            input_1.append(images[j])
            input_1.append(images[k])

        input_1 = np.array(input_1)
        input = [input_1, input_1, input_1]
        yield(input, np.zeros((batch_size, )))

In [54]:
import numpy as np
from pandas import HDFStore,DataFrame# create (or open) an hdf5 file and opens in append mode
hdf = HDFStore('Train/triplet_weight_final.hdf5')

In [55]:
from keras.models import load_model

print("Loading pre-trained weight")
deep_rank_model.load_weights('Train/triplet_weight_final.hdf5')


Loading pre-trained weight


OSError: Unable to open file (truncated file: eof = 96, sblock->base_addr = 0, stored_eof = 2048)

In [56]:
from keras.callbacks import ModelCheckpoint

filepath = 'Train/triplet_weight_final.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [57]:
X,y

(array([[[[0.8627451 , 0.7254902 , 0.57647059],
          [0.8627451 , 0.7254902 , 0.57254902],
          [0.86666667, 0.72156863, 0.56862745],
          ...,
          [0.67843137, 0.50196078, 0.34901961],
          [0.63921569, 0.45098039, 0.30196078],
          [0.62352941, 0.43529412, 0.28627451]],
 
         [[0.85882353, 0.72156863, 0.57254902],
          [0.8627451 , 0.72156863, 0.56862745],
          [0.86666667, 0.71764706, 0.56470588],
          ...,
          [0.68235294, 0.50196078, 0.35294118],
          [0.63921569, 0.45098039, 0.30588235],
          [0.62352941, 0.43529412, 0.28627451]],
 
         [[0.85882353, 0.71372549, 0.56078431],
          [0.85882353, 0.71372549, 0.56078431],
          [0.8627451 , 0.70980392, 0.55686275],
          ...,
          [0.68235294, 0.50980392, 0.36078431],
          [0.63921569, 0.45882353, 0.30980392],
          [0.62352941, 0.43921569, 0.29411765]],
 
         ...,
 
         [[0.16470588, 0.12156863, 0.14509804],
          [0.16470

In [60]:
import time    
start_time = time.time()
deep_rank_model.fit_generator(generator=image_batch_generator(X, y, batch_size),
                   steps_per_epoch=len(X)//batch_size,
                   epochs=2000,
                   verbose=1,
                   callbacks=callbacks_list)


end_time = time.time()

Epoch 1/2000


InvalidArgumentError:  Input to reshape is a tensor with 147456 values, but the requested shape requires a multiple of 4704
	 [[node model_7/flatten_6/Reshape (defined at <ipython-input-58-c5b272690cab>:3) ]] [Op:__inference_train_function_8441]

Function call stack:
train_function
